# AlexNet com recursos limitados

Antes implementamos a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) sem considerar a questão de quantidade de parâmetros. Nesta prática, vamos focar nesse quesito.






## Configuração do ambiente

In [1]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary

import time, os, sys, numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

## Carregamento das bases de dados

A função `load_data_cifar10` carrega e prepara o dataset CIFAR-10 para treinamento e teste.

A função `load_data_fashion_mnist` faz o mesmo para o dataset Fashion MNIST. Esse conjunto de dados apresenta imagens em tons de cinza com dimensão de 28 x 28, e também é dividido em 10 classes.

In [3]:
def load_data_cifar10(batch_size, resize=None):
    root = '/pgeoprj2/ciag2024/dados/cifar/'

    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    cifar10_train = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transformer)
    cifar10_test = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 1

    train_iter = torch.utils.data.DataLoader(cifar10_train,
                                            batch_size, shuffle=True,
                                            num_workers=num_workers)

    test_iter = torch.utils.data.DataLoader(cifar10_test,
                                            batch_size, shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter

def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join('~', '.pytorch', 'datasets', 'fashion-mnist')):
    root = os.path.expanduser(root)

    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True,download=True,transform=transformer)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False,download=True,transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 1

    train_iter = torch.utils.data.DataLoader(mnist_train,
                                            batch_size, shuffle=True,
                                            num_workers=num_workers)

    test_iter = torch.utils.data.DataLoader(mnist_test,
                                            batch_size, shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter

## Funções auxiliares

* `_get_batch` retorna as features e os labels de um batch

* `evaluate_accuracy` calcula a acurácia de um modelo em um dataset.

* `train_validate` implemneta o treinamento e validação de uma rede.

In [4]:
def _get_batch(batch):
    features, labels = batch

    if labels.type() != features.type():
        labels = labels.type(features.type())

    return (torch.nn.DataParallel(features, device_ids=devices_ids),
            torch.nn.DataParallel(labels, device_ids=devices_ids), features.shape[0])

def evaluate_accuracy(data_iter, net, loss):
    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs):
    print('training on', device)

    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)

            trainer.zero_grad()
            l = loss(y_hat, y).sum()

            l.backward()
            trainer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]

        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)

        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))

## AlexNet

<p align="center">
  <img width=700 src="https://miro.medium.com/max/700/1*vXBvV_Unz3JAxytc5iSeoQ.png">
</p>

A AlexNet foi uma arquitetura muito famosa. Entretanto, ela possui muitos parâmetros.
Essa arquitetura, para classificar 10 classes, tem um total de **58.312.736** parâmetros, como mostrado na tabela abaixo.

**Camada** | **Calc Parâmetros** | **Total Parâmetros**
--- | ---: | ---:
Convolução 1 | 11\*11\*3\*96 | 34.848
Convolução 2 | 5\*5\*96\*256 | 614.400
Convolução 3 | 3\*3\*256\*384 | 884.736
Convolução 4 | 3\*3\*384\*384 | 1.327.104
Convolução 5 | 3\*3\*384\*256 | 884.736
FC 6 | 9216*4096 | 37.748.736
FC 7 | 4096*4096 | 16.777.216
FC 8 | 4096*10 | 40.960
**Total** | | **58.312.736**


### Rede original (para efeito de comparação)

In [5]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),

            nn.Linear(9216, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [6]:
num_epochs, lr, batch_size, weight_decay = 20, 0.001, 100, 0.0001

net = AlexNet(input_channels=3, classes=10)
net.to(device)
print(summary(net,(3,227,227)))

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

### Atividade - Arquitetura usando batch normalization e convolução dilatada

O objetivo nessa prática é adaptar a rede neural proposta, baseada na [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), e incluir [*batch normalization*](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) e camadas de [convolução dilatada](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.htmll).

> Inclua uma operação de batch normalization em cada uma das camadas convolucionais e nas duas primeiras transformações lineares.

- Atente-se para especificar se o batch normalization é 1D ([nn.BatchNorm1d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html)) ou 2D ([nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)).
- Normalmente, aplicamos a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167) entre a transformação e a função de ativação em uma camada densa.
- Para camadas convolucionais, o [*batch normalization*](https://arxiv.org/abs/1502.03167) ocorre após o cálculo da convolução e antes da aplicação da função de ativação.
- Nas funções de batch normalization, o parâmetro num_features será a quantidade de canais de saída da convolução, se a normalização estiver sendo aplicada em uma camada convolucional ou será a quantidade de features de saída da camada linear, se linear.

> Inclua a convolução dilatada na camada conv2 e conv5 por meio do parâmetro dilation da função Conv2d.

- Para a camada conv2, use uma taxa de dilatação de 2
- Para a camada conv5, use uma taxa de dilatação de 4
- Neste caso, como usamos filtros dilatados em duas camadas (com dilatação aumentante, ou seja, dilatação 2 seguida da dilatação 4), remova (ou comente com #) duas camadas convolucionais (conv3 e conv4), já que o *receptive field* se mantem similar dessa forma.

> Ao final, observe que mesmo com uma quantidade menor de parâmetros, a nova rede alcança uma acurácia similar ou melhor que a rede original vista na prática passada.

In [7]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0),

            # nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(num_features=384),
            # nn.ReLU(),

            # nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            # nn.BatchNorm2d(num_features=384),
            # nn.ReLU(),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, dilation=4),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),

            nn.Linear(1024, 4096),
            nn.BatchNorm1d(num_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.BatchNorm1d(num_features=4096),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(4096, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [8]:
num_epochs, lr, batch_size, weight_decay = 20, 0.001, 100, 0.0001

net = AlexNet(input_channels=3, classes=10)
net.to(device)
print(summary(net,(3,227,227)))

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)
# train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=227)

trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
       BatchNorm2d-2           [-1, 96, 55, 55]             192
              ReLU-3           [-1, 96, 55, 55]               0
         MaxPool2d-4           [-1, 96, 27, 27]               0
            Conv2d-5          [-1, 256, 23, 23]         614,656
       BatchNorm2d-6          [-1, 256, 23, 23]             512
              ReLU-7          [-1, 256, 23, 23]               0
         MaxPool2d-8          [-1, 256, 11, 11]               0
            Conv2d-9            [-1, 256, 5, 5]         590,080
      BatchNorm2d-10            [-1, 256, 5, 5]             512
             ReLU-11            [-1, 256, 5, 5]               0
        MaxPool2d-12            [-1, 256, 2, 2]               0
          Flatten-13                 [-1, 1024]               0
           Linear-14                 [-

.